In [ ]:
# import os
# import cv2
#
# # this is for processing UR data video size, ignore it
# # Paths
# input_folder = './fall_videos/ur_videos'
# output_folder = './fall_videos/processed'
#
# # Loop through each video in the input folder
# for filename in os.listdir(input_folder):
#     if filename.endswith('.mp4'):
#         input_path = os.path.join(input_folder, filename)
#         output_path = os.path.join(output_folder, filename)
#
#         cap = cv2.VideoCapture(input_path)
#
#         # Get original properties
#         fps = cap.get(cv2.CAP_PROP_FPS)
#         width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#         height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#
#         # Output: half the width (right side)
#         fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#         out = cv2.VideoWriter(output_path, fourcc, fps, (width // 2, height))
#
#         print(f"Processing {filename}...")
#
#         while True:
#             ret, frame = cap.read()
#             if not ret:
#                 break
#
#             # Crop right half: frame[:, 320:] if original width = 640
#             right_half = frame[:, width // 2:]
#
#             out.write(right_half)
#
#         cap.release()
#         out.release()
#         print(f"Saved to {output_path}")

In [4]:
from ultralytics import YOLO
import pandas as pd
import os
import cv2

# I run this because run jupiter on this cause crash

# GENERATE keypoints here
model = YOLO("yolo11n-pose.pt")  # load an official model

input_folder = './fall_videos/ur_videos_processed'

all_keypoints = []

def is_none_or_empty(x):
    return x is None or len(x) == 0

# normalize data according to the size of videos in different dataset
def normalize(list, width, height, offset):
    for i in range(offset, 51 + offset, 3):  # 17 keypoints, each keypoint is x, y and confidence
        list[i] = float(list[i]) / width
        list[i + 1] = float(list[i + 1]) / height

for filename in os.listdir(input_folder):
    if filename.endswith('.mp4'):
        full_path = os.path.join(input_folder, filename)
        print("Processing {}".format(full_path))
        results = model.track(source=full_path, stream=True)

        for frame_number, result in enumerate(results): # each is a frame
            height, width = result.orig_img.shape[:2]
            boxes = result.boxes  # Detections
            keypoints = result.keypoints  # If using a pose model

            if is_none_or_empty(boxes) or is_none_or_empty(keypoints):
                continue

            best_index = None
            best_conf = 0.0

            # inside a frame can have multiple people but only take the highest confidence
            for i in range(len(boxes)):
                conf = float(boxes.conf[i])
                cls = int(boxes.cls[i])
                if cls == 0 and conf > best_conf:
                    best_conf = conf
                    best_index = i

            # print("best confidence: ", best_conf)

            # keypoints.data is a torch.Tensor of shape (num_detections, num_keypoints, 3)
            keypoints_tensor = keypoints.data[best_index]

            keypoints_np = keypoints_tensor.cpu().detach().numpy()

            base_name = os.path.splitext(filename)[0]
            tag = f"{base_name}_{frame_number}"

            flat = keypoints_np.flatten().tolist()
            normalize(flat, width, height, 0)

            # Extract bounding box coordinates
            bbox = boxes[best_index].xyxy.squeeze(0).tolist()  # [xmin, ymin, xmax, ymax]

            # Combine with flattened keypoints and tag
            flat_with_tag = bbox + flat + [tag] # videoname_frame

            # Store the result
            all_keypoints.append(flat_with_tag)

df = pd.DataFrame(all_keypoints)
df.to_csv('./csv_data/ur_keypoints.csv', index=False, header=False)

Processing ./fall_videos/processed\adl-01-cam0.mp4

video 1/1 (frame 1/150) C:\Users\sonnpm\OneDrive - UTS\Desktop\UTS\Semester 3\DL\Assignment\ass 3\fall_videos\processed\adl-01-cam0.mp4: 480x640 (no detections), 85.5ms
video 1/1 (frame 2/150) C:\Users\sonnpm\OneDrive - UTS\Desktop\UTS\Semester 3\DL\Assignment\ass 3\fall_videos\processed\adl-01-cam0.mp4: 480x640 1 person, 8.4ms
video 1/1 (frame 3/150) C:\Users\sonnpm\OneDrive - UTS\Desktop\UTS\Semester 3\DL\Assignment\ass 3\fall_videos\processed\adl-01-cam0.mp4: 480x640 1 person, 10.0ms
video 1/1 (frame 4/150) C:\Users\sonnpm\OneDrive - UTS\Desktop\UTS\Semester 3\DL\Assignment\ass 3\fall_videos\processed\adl-01-cam0.mp4: 480x640 1 person, 9.4ms
video 1/1 (frame 5/150) C:\Users\sonnpm\OneDrive - UTS\Desktop\UTS\Semester 3\DL\Assignment\ass 3\fall_videos\processed\adl-01-cam0.mp4: 480x640 1 person, 8.2ms
video 1/1 (frame 6/150) C:\Users\sonnpm\OneDrive - UTS\Desktop\UTS\Semester 3\DL\Assignment\ass 3\fall_videos\processed\adl-01-cam0.mp

In [5]:
import csv
import pandas as pd

# Load label.csv and create a lookup dictionary
label_df = pd.read_csv("./csv_data/ur_label.csv", header=None, names=["videoname", "framenumber", "label"])
label_dict = {
    f"{row['videoname']}_{row['framenumber']}": row["label"]
    for _, row in label_df.iterrows()
}

print(label_dict)

{'fall-01_1': 0, 'fall-01_2': 0, 'fall-01_3': 0, 'fall-01_4': 0, 'fall-01_5': 0, 'fall-01_6': 0, 'fall-01_7': 0, 'fall-01_8': 0, 'fall-01_9': 0, 'fall-01_10': 0, 'fall-01_11': 0, 'fall-01_12': 0, 'fall-01_13': 0, 'fall-01_14': 0, 'fall-01_15': 0, 'fall-01_16': 0, 'fall-01_17': 0, 'fall-01_18': 0, 'fall-01_19': 0, 'fall-01_20': 0, 'fall-01_21': 0, 'fall-01_22': 0, 'fall-01_23': 0, 'fall-01_24': 0, 'fall-01_25': 0, 'fall-01_26': 0, 'fall-01_27': 0, 'fall-01_28': 0, 'fall-01_29': 0, 'fall-01_30': 0, 'fall-01_31': 0, 'fall-01_32': 0, 'fall-01_33': 0, 'fall-01_34': 0, 'fall-01_35': 0, 'fall-01_36': 0, 'fall-01_37': 0, 'fall-01_38': 0, 'fall-01_39': 0, 'fall-01_40': 0, 'fall-01_41': 0, 'fall-01_42': 0, 'fall-01_43': 0, 'fall-01_44': 0, 'fall-01_45': 0, 'fall-01_46': 0, 'fall-01_47': 0, 'fall-01_48': 0, 'fall-01_49': 0, 'fall-01_50': 0, 'fall-01_51': 0, 'fall-01_52': 0, 'fall-01_53': 0, 'fall-01_54': 0, 'fall-01_55': 0, 'fall-01_56': 0, 'fall-01_57': 0, 'fall-01_58': 0, 'fall-01_59': 0, 'fall

In [8]:
import re

unknown_cnt = 0
with open("./csv_data/ur_keypoints.csv", "r") as data_file, open("./csv_data/ur_keypoints_labeled.csv", "w", newline="") as out_file:
    reader = csv.reader(data_file)
    writer = csv.writer(out_file)

    for row in reader:
        # The last column is assumed to be videoname_framenumber
        key = row[-1]
        key = re.sub(r"-cam\d+", "", key)
        label = label_dict.get(key)

        if label is not None:
            new_row = [label] + row  # prepend label
            writer.writerow(new_row)
        else:
            unknown_cnt = unknown_cnt + 1
            print(f"Warning: No label found for {key}, skipping row.")

print(unknown_cnt)

162
